In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import pandas as pd
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json
import collections
import time
from gspread_formatting import *

In [ ]:
# Authenticating the credentials for google sheets
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)

In [ ]:
# Get all the grants from the Portal
my_key = get_key('default')
all_grants = ff_utils.search_metadata('/search/?type=Award', key=my_key)

#Get all the centers info from google management sheets
gs_management_sheet_id = ''
gs_management_workbook = gc.open_by_key(gs_management_sheet_id)
gs_management_worksheet = gs_management_workbook.sheet1

centers_sheets_links = gs_management_worksheet.get_all_values()

grants = []
for center in centers_sheets_links:
    if 'Guttman' in center[0]:
        guttman_grants = center[1].split(',')
        for guttman_grant in guttman_grants:
            for grant in all_grants:
                if guttman_grant in grant['@id']:
                    grants.append(grant)
    else:
        for grant in all_grants:
            if center[1] in grant['@id']:
                grants.append(grant)

# Add Petronis grant
for grant in all_grants:
    if grant['uuid'] == '2850ac0b-92f4-4c8b-8f6a-5199afc442b1':
        grants.append(grant)

print(len(grants))

In [ ]:
all_grants[23]['@id']

In [ ]:
# Create Data Plan Tables
ct = 0
for grant in grants:
    # for testing only on a specific center
#     if grant['uuid'] != 'ae6c618f-7a8c-441e-a886-e30bbbe591da':
#         continue
    # Guttman NT award, ignore for now
    if grant['uuid'] == '8a9d6db4-37eb-488e-828c-d90f3ab348f6':
        continue
    ct = ct + 1
    grant_uuid = grant['uuid']
    q = '/search/?type=ExperimentSetReplicate&award.uuid={}'.format(grant_uuid)
    exps_in_grant = ff_utils.search_metadata(q, key=my_key)

    if len(exps_in_grant) == 0:
        header_message = 'There have been no data submissions from your center yet. Please add rows below for expected submission through August 2020. Please see link below for more examples.'
        link_template = '=hyperlink("https://docs.google.com/spreadsheets/d/1BbB2p20wrnlc1isiGYp5_-F6BmHCHNDREm7k_CzNQrc/edit#gid=0","examples")'
        tail_message1 = ['add planned submissions here!', '', '', '', '', '', '', '','','']
        tail_message2 = ['example: Alice Allison, UNI', 'hi-c variation project', 'in situ Hi-C', 'U2OS', 'hi-c protocol variatons on h1', '5', 'planned', '2019-12-20', '','']
        tail_message3 = ['example: Bob Baker, INST', 'ctcf tracking project', 'SPT', 'U2OS', 'CTCF tracking on U2OS', '30', 'planned', '2020-03-31','','']

        for center in centers_sheets_links:
            if 'Guttman' in center[0]:
                guttman_grants = center[0].split(',')
                for guttman_grant in guttman_grants:
                    if guttman_grant in grant['@id']:
                        center_gs_id = center[2]
            else:
                if center[1] in grant['@id']:
                    center_gs_id = center[2]

        workbook_id = center_gs_id
        workbook = gc.open_by_key(workbook_id)

        print('working on', workbook)

        try:
            worksheet = workbook.add_worksheet('Data Status/Plans', 100, 20)

        except:
            worksheet = workbook.worksheet('Data Status/Plans')

        gs_write = []
        gs_write.append(gspread.models.Cell(1,1,header_message))
        gs_write.append(gspread.models.Cell(2,1, link_template))
        row = 3
        col = 1
        col_labels = ['Lab', 'Publication/Project', 'Experiment Type', 'Biosample', 'Dataset','Num Experiment Sets', 'Status of release', 'Public release date', 'Project release date', 'Browser Link']
        for i in col_labels:
            gs_write.append(gspread.models.Cell(row,col,i))
            col = col + 1
        col = 1
        
        for txt in tail_message1:
            gs_write.append(gspread.models.Cell(row + 1, col, txt))
            col = col + 1
        col = 1
        for txt in tail_message2:
            gs_write.append(gspread.models.Cell(row + 2, col, txt))
            col = col + 1
        col = 1
        for txt in tail_message3:
            gs_write.append(gspread.models.Cell(row + 3, col, txt))
            col = col + 1

        
        worksheet.update_cells(gs_write,value_input_option='USER_ENTERED')
        fmt1 = cellFormat(textFormat=textFormat(foregroundColor=color(0.98,0.01,0.01)))
        format_cell_ranges(worksheet, [('A{}:J{}'.format(row+1, row+3), fmt1)])
        fmt2 = cellFormat(textFormat=textFormat(bold=True))
        format_cell_ranges(worksheet, [('A1:J3', fmt2)])
        if ct == 10:
            time.sleep(100)
            ct = 0
    else:
        header_message = 'Below is the list of data submitted by your center so far. Please add rows below for expected submission through August 2020. Please see link below for more examples.'
        link_template = '=hyperlink("https://docs.google.com/spreadsheets/d/1BbB2p20wrnlc1isiGYp5_-F6BmHCHNDREm7k_CzNQrc/edit#gid=0","examples")'
        tail_message1 = ['add planned submissions here!', '', '', '', '', '', '', '', '', '']
        tail_message2 = ['example: Alice Allison, UNI', 'hi-c variation project', 'in situ Hi-C', 'U2OS', 'hi-c protocol variatons on h1', '5', 'planned', '2019-12-20', '', '']
        tail_message3 = ['example: Bob Baker, INST', 'ctcf tracking project', 'SPT', 'U2OS', 'CTCF tracking on U2OS', '30', 'planned', '2020-03-31', '', '']

        print(grant['@id'])
        print(len(exps_in_grant))

        # Getting the information from each experiment set and group them in same dataset
        my_dict = {}
        my_dict = collections.defaultdict(dict)
        for exp_replicate_set in exps_in_grant:
            exp_replicate_set_keys = exp_replicate_set.keys()
            lab = exp_replicate_set['lab']['display_title']
            publication = exp_replicate_set['publications_of_set']

            if publication:
                publication_title = [publication[x]['display_title'][:publication[x]['display_title'].index(")") + 1] for x in range(len(publication))]
                if len(publication_title) == 1:
                    publication_title = publication_title[0]
            else:
                publication_title = ''

            experiments_in_set = exp_replicate_set['experiments_in_set']
            if len(experiments_in_set) == 0:
                continue
            biosource = experiments_in_set[0]['biosample']['biosource_summary']
            status = exp_replicate_set['status']
            project_release_date = exp_replicate_set['project_release'] if 'project_release' in exp_replicate_set_keys else ''
            exp_type = experiments_in_set[0]['experiment_type']['display_title']

            dataset_title_iskey = False
            if 'dataset_label' in exp_replicate_set_keys:
                dataset_title = exp_replicate_set['dataset_label']
                dataset_key = exp_replicate_set['dataset_label'] + '/' + exp_type
                dataset_title_iskey = True

            else:
                dataset_title = exp_type + ' on ' + biosource
                dataset_key = dataset_title

            public_release_date = exp_replicate_set['public_release'] if 'public_release' in exp_replicate_set_keys else ''
            if dataset_key not in my_dict.keys():
                my_dict[dataset_key] = collections.defaultdict(dict)
            
            my_dict[dataset_key][exp_replicate_set['uuid']] = {'lab':lab, 'publication': publication_title, 'experiment_type': exp_type, 'biosample':biosource,'dataset_title': dataset_title, 'public_release_date':public_release_date, 'project_release_date': project_release_date, 'status': status, 'dataset_title_iskey':dataset_title_iskey}
            
        #Compare experiment sets for each dataset
        for dataset_key, dataset in my_dict.items():
            dataset_label = collections.defaultdict(dict)
            indx = 0
            num_biosample = 0
            num_status = 0
            for uuid, exp_set in dataset.items():
                if indx == 0:
                    dataset_label['lab'] = exp_set['lab']
                    dataset_label['publication'] =  exp_set['publication']
                    dataset_label['experiment_type'] = exp_set['experiment_type']
                    dataset_label['biosample'] = exp_set['biosample']
                    dataset_label['dataset_title'] = exp_set['dataset_title']
                    dataset_label['num_exp_set'] = len(dataset.keys())
                    dataset_label['status'] = exp_set['status']
                    dataset_label['public_release_date'] = exp_set['public_release_date']
                    dataset_label['project_release_date'] = exp_set['project_release_date']
                    dataset_label['dataset_title_iskey'] = exp_set['dataset_title_iskey']
                    indx = indx + 1
                    continue
                for key in dataset_label:
                    if key == 'num_exp_set':
                        continue
                    if exp_set[key] != dataset_label[key]:
                        if key == 'project_release_date' or key == 'public_release_date':
                            if dataset_label[key] < exp_set[key]:
                                dataset_label[key] = exp_set[key]
                                continue
                        elif key == 'biosample':
                            num_biosample = num_biosample + 1
                        elif key == 'status':
                            num_status = num_status + 1
                        else:
                            print(key, dataset_key)
                            print(exp_set[key],dataset_label[key])

            if dataset_label['dataset_title_iskey']:
                browser_link = 'https://data.4dnucleome.org/browse/?award.uuid={}&dataset_label={}&experiments_in_set.experiment_type={}'.format(grant_uuid, dataset_label['dataset_title'], dataset_label['experiment_type'])
                browser_link_fotmat = '=HYPERLINK("{}", "View in Portal")'.format(browser_link)
            else:
                browser_link = "https://data.4dnucleome.org/browse/?award.uuid={}&dataset_label=No+value&experiments_in_set.biosample.biosource_summary={}&experiments_in_set.experiment_type={}".format(grant_uuid, dataset_label['biosample'], dataset_label['experiment_type'])
                browser_link_fotmat = '=HYPERLINK("{}", "View in Portal")'.format(browser_link)
                dataset_label['dataset_title'] = ''

            dataset_label.pop('dataset_title_iskey')
            dataset_label['browser_view'] = browser_link_fotmat

            if num_biosample > 0:
                dataset_label['biosample'] = dataset_label['biosample'] + " and {} other(s)".format(num_biosample)

            if num_status > 0:
                dataset_label['status'] = 'Partially released'

            my_dict[dataset_key]['out_info'] = dataset_label

        # Sorting datasets by Publication, Lab and then Experiment Type
        list_datasets = []
        for k in my_dict:
            interest = my_dict[k]["out_info"]
            temp = {}
            temp['dataset_label'] = k
            for k_sub, v_sub in interest.items():
                temp[k_sub] = v_sub
            list_datasets.append(temp)
        list_datasets = sorted(list_datasets, key=lambda x: (x['publication'],x['lab'], x['experiment_type'] ))    

        for center in centers_sheets_links:
            if 'Guttman' in center[0]:
                guttman_grants = center[1].split(',')
                for guttman_grant in guttman_grants:
                    if guttman_grant in grant['@id']:
                        grant_gs_id = center[2]
            if center[1] in grant['@id']:
                grant_gs_id = center[2]
                
        workbook_id = grant_gs_id 
        workbook = gc.open_by_key(workbook_id)
        print('working on', workbook)

        try:
            worksheet = workbook.add_worksheet('Data Status/Plans', 100, 20)

        except:
            worksheet = workbook.worksheet('Data Status/Plans')

        gs_write = []
        gs_write.append(gspread.models.Cell(1,1,header_message))
        gs_write.append(gspread.models.Cell(2,1, link_template))
        row = 3
        col = 1
        col_labels = ['Lab', 'Publication/Project', 'Experiment Type', 'Biosample', 'Dataset','Num Experiment Sets', 'Status of release', 'Public release date', 'Project release date', 'Browser Link']
        for col_label in col_labels:
            gs_write.append(gspread.models.Cell(row, col, col_label))
            col = col + 1
        col = 0
        for dataset in list_datasets:
            row = row + 1
            dataset.pop('dataset_label')
            for key in dataset:
                col = col + 1
                gs_write.append(gspread.models.Cell(row,col,dataset[key]))
            col = 0 
        col = 1
        
        for txt in tail_message1:
            gs_write.append(gspread.models.Cell(row + 1, col, txt))
            col = col + 1

        col = 1
        for txt in tail_message2:
            gs_write.append(gspread.models.Cell(row + 2, col, txt))
            col = col + 1  
            
        col = 1
        for txt in tail_message3:
            gs_write.append(gspread.models.Cell(row + 3, col, txt))
            col = col + 1 
            
            
        
        worksheet.update_cells(gs_write,value_input_option='USER_ENTERED')
        
        if ct == 10:
            time.sleep(100)
            ct = 0

        # Add formatting to the google sheet
        fmt1 = cellFormat(textFormat=textFormat(foregroundColor=color(0.98,0.01,0.01))) # color red
        fmt2 = cellFormat(textFormat=textFormat(bold=True))
        format_cell_ranges(worksheet, [('A{}:J{}'.format(row+1, row+3), fmt1)])
        format_cell_ranges(worksheet, [('A1:J3', fmt2)])

In [ ]:
experiments_in_set[0]['experiment_type']

In [ ]:
print(exps_in_grant[0]['uuid'])

In [ ]:
len(exp_replicate_set['experiments_in_set'])

In [ ]:
print(len(grants))

In [ ]:
print(grants[0].keys())